In [1]:
!neofetch

/usr/bin/sh: 1: neofetch: not found


In [2]:
import os
from os.path import join as J
import pandas as pd
import numpy as np
import subprocess

In [3]:
data_path = "~/work/ai-academy2020/data/"

In [4]:
!tree {data_path}

/usr/bin/sh: 1: tree: not found


In [5]:
pd.read_csv( J(data_path, "small_group_description.csv") ).head()

,small_group,small_group_code
0,Зоотовары,0
1,Сетевые супермаркеты и продуктовые магазины,1
2,Обслуживание существующего автомобиля (АЗС),2
3,Оплата телефона и связи,3
4,Аптеки,4


In [6]:
transactions_train = pd.read_csv('data/transactions_train.csv')

In [7]:
transactions_train.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [8]:
transactions_train["trans_date"]

0             6
1             6
2             8
3             9
4            10
           ... 
26450572    727
26450573    727
26450574    727
26450575    727
26450576    729
Name: trans_date, Length: 26450577, dtype: int64

In [9]:
def prep_date(data):
    data["trans_month"] = ((data["trans_date"] // 30) % 12 + 1)
    data["trans_year"] = ((data["trans_date"] // 365) + 1)

In [10]:
prep_date(transactions_train)
transactions_train.trans_year.value_counts()

2    14682465
1    11768112
Name: trans_year, dtype: int64

* client_id - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* small_group - категория покупки
* amount_rur - сумма транзакции

In [11]:
transactions_train.head(1)

,client_id,trans_date,small_group,amount_rur,trans_month,trans_year
0,33172,6,4,71.463,1,1


In [12]:
def get_first_last_transaction_date(data):
    ret = data.groupby('client_id')["trans_date"].agg(["min", "max"])
    ret.columns = ["first_transaction_date", "last_transaction_date"]
    return ret

In [13]:
def get_minmax_yearly_trans(data):
    ret = data.groupby(['client_id','trans_year'])["amount_rur"].agg(['count', 'sum'])\
        .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    ret.columns = ['client_id', 'yeartrans_max', 'yeartrans_min', 'yearcount_max', 'yearcount_min']
    ret["yearcount_diff"] = ret["yearcount_max"] - ret["yearcount_min"]
    return ret
get_minmax_yearly_trans(transactions_train)

,client_id,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff
0,4,363,357,15264.437,13139.684,2124.753
1,6,373,357,7978.615,7742.124,236.491
2,7,482,291,27803.883,25826.153,1977.730
3,10,411,295,20585.312,13834.053,6751.259
4,11,450,362,18019.155,8770.249,9248.906
...,...,...,...,...,...,...
29995,49993,515,386,14364.269,9883.275,4480.994
29996,49995,533,436,15331.092,12620.064,2711.028
29997,49996,602,536,44188.509,36711.836,7476.673
29998,49997,488,227,8125.031,5168.084,2956.947


In [14]:
def get_minmax_monthly_trans(data):
    ret = data.groupby(['client_id','trans_month'])["amount_rur"].agg(['count', 'sum'])\
        .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    ret.columns = ['client_id', 'monthtrans_max', 'monthtrans_min', 'monthcount_max', 'monthcount_min']
    ret["monthcount_diff"] = ret["monthcount_max"] - ret["monthcount_min"]
    return ret
get_minmax_monthly_trans(transactions_train)

,client_id,monthtrans_max,monthtrans_min,monthcount_max,monthcount_min,monthcount_diff
0,4,66,52,3636.275,1658.041,1978.234
1,6,82,42,2178.090,863.566,1314.524
2,7,74,28,10053.767,1666.168,8387.599
3,10,88,42,5169.318,1513.850,3655.468
4,11,91,56,3966.888,936.898,3029.990
...,...,...,...,...,...,...
29995,49993,94,58,3013.794,1495.437,1518.357
29996,49995,123,62,4270.710,1342.261,2928.449
29997,49996,132,62,12893.703,2107.605,10786.098
29998,49997,91,21,1630.802,524.260,1106.542


In [15]:
def get_row_with_max_col_val(df, col, group_col):
    idx = df.groupby(group_col)[col].transform(max) == df[col]
    return df[idx]

In [16]:
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [17]:
max_group = get_row_with_max_col_val(transactions_train, 'amount_rur', 'client_id')[ ["client_id", "small_group"] ]

In [18]:
agg_features.head()

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


In [19]:
counter_df_train=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()

In [20]:
sum_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].agg(["sum","max"])

In [21]:
sum_df_train.head(1)

,,sum,max
client_id,small_group,,
4,1,15049.897,265.798


# Посмотрим на максимальные - минимальные затраты за 1 день

In [23]:
transactions_train

,client_id,trans_date,small_group,amount_rur,trans_month,trans_year
0,33172,6,4,71.463,1,1
1,33172,6,35,45.017,1,1
2,33172,8,11,13.887,1,1
3,33172,9,11,15.983,1,1
4,33172,10,11,21.341,1,1
...,...,...,...,...,...,...
26450572,43300,727,25,7.602,1,2
26450573,43300,727,15,3.709,1,2
26450574,43300,727,1,6.448,1,2
26450575,43300,727,11,24.669,1,2


In [24]:
transactions_train.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum']).agg(['max', 'min'])

,count,sum
max,14,413371.598
min,1,0.001


In [25]:
minmax_dayly_transactions_train = \
transactions_train.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum'])\
.groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

minmax_dayly_transactions_train.columns = ['client_id', 'daytrans_max', 'daytrans_min', 'daycount_max', 'daycount_min']

In [26]:
cat_counts_train=counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_counts_train=cat_counts_train.fillna(0)
cat_counts_train.columns=['sg_count_'+str(i) for i in cat_counts_train.columns]

In [27]:
cat_sum_train = sum_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values=["sum","max"])
cat_sum_train = cat_sum_train.fillna(0)
cat_sum_train.columns=['sg_sum_'+str(i) for i in cat_sum_train.columns]

In [28]:
cat_sum_train.head()

,"sg_sum_('sum', 0)","sg_sum_('sum', 1)","sg_sum_('sum', 2)","sg_sum_('sum', 3)","sg_sum_('sum', 4)","sg_sum_('sum', 5)","sg_sum_('sum', 6)","sg_sum_('sum', 7)","sg_sum_('sum', 8)","sg_sum_('sum', 9)",...,"sg_sum_('max', 192)","sg_sum_('max', 193)","sg_sum_('max', 195)","sg_sum_('max', 196)","sg_sum_('max', 197)","sg_sum_('max', 198)","sg_sum_('max', 199)","sg_sum_('max', 200)","sg_sum_('max', 202)","sg_sum_('max', 203)"
client_id,,,,,,,,,,,,,,,,,,,,,
4,0.000,15049.897,98.966,835.403,3799.572,0.000,0.0,0.000,22.852,514.861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,53.372,9602.770,0.000,2905.083,189.480,0.000,0.0,0.000,0.000,80.193,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,89.072,4632.372,125.521,888.523,660.002,1241.557,0.0,118.975,197.820,3568.778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,639.798,13440.506,24.928,2034.764,2422.679,0.000,0.0,0.000,270.330,722.931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,88.239,5037.288,0.000,637.526,178.423,1844.468,0.0,0.000,0.000,86.990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
minmax_dayly_transactions_train

,client_id,daytrans_max,daytrans_min,daycount_max,daycount_min
0,4,5,1,1341.802,1.282
1,6,4,1,412.967,2.290
2,7,7,1,4534.593,0.065
3,10,7,1,792.126,1.221
4,11,4,1,2112.019,0.458
...,...,...,...,...,...
29995,49993,5,1,1325.683,1.897
29996,49995,7,1,1243.601,0.690
29997,49996,8,1,2245.405,4.270
29998,49997,5,1,863.182,1.509


In [30]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


def experiment():
    y_train = pd.read_csv( J(data_path, "train_target.csv") )
    train=pd.merge(y_train,cat_counts_train,on='client_id')
    

In [31]:
y_train

NameError: name 'y_train' is not defined

In [32]:
agg_features

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058
...,...,...,...,...,...,...
29995,49993,24247.544,26.911814,73.592787,0.211,1315.470
29996,49995,27951.156,28.845362,64.723186,0.690,1243.601
29997,49996,80900.345,71.089934,125.642727,0.458,1657.546
29998,49997,13293.115,18.591769,38.841011,0.432,858.240


In [33]:
minmax_dayly_transactions_train

,client_id,daytrans_max,daytrans_min,daycount_max,daycount_min
0,4,5,1,1341.802,1.282
1,6,4,1,412.967,2.290
2,7,7,1,4534.593,0.065
3,10,7,1,792.126,1.221
4,11,4,1,2112.019,0.458
...,...,...,...,...,...
29995,49993,5,1,1325.683,1.897
29996,49995,7,1,1243.601,0.690
29997,49996,8,1,2245.405,4.270
29998,49997,5,1,863.182,1.509


In [34]:
y_train = pd.read_csv("data/train_target.csv")
train=pd.merge(y_train,agg_features,on='client_id')
train=pd.merge(train,cat_counts_train.reset_index(),on='client_id')
train=pd.merge(train,cat_sum_train.reset_index(),on='client_id')

train=pd.merge(train,max_group.reset_index()[["client_id", "small_group"]],on='client_id')
train=pd.merge(train,minmax_dayly_transactions_train,on='client_id')

In [35]:
train=pd.merge(train,get_minmax_monthly_trans(transactions_train),on='client_id')
train=pd.merge(train,get_minmax_yearly_trans(transactions_train),on='client_id')

In [36]:
train=pd.merge(train,get_first_last_transaction_date(transactions_train).reset_index(),on='client_id')

In [37]:
train

,client_id,bins,sum,mean,std,min,max,sg_count_0,sg_count_1,sg_count_2,...,monthcount_max,monthcount_min,monthcount_diff,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff,first_transaction_date,last_transaction_date
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,...,5299.353,1335.528,3963.825,444,426,15295.465,14958.546,336.919,0,695
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,...,5311.449,2241.328,3070.121,481,337,22440.602,20107.968,2332.634,0,729
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,...,3749.132,1431.592,2317.540,420,362,18470.716,8372.100,10098.616,4,729
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,...,2023.154,720.864,1302.290,538,438,7994.095,7779.031,215.064,10,729
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,...,1998.422,351.017,1647.405,399,385,7436.374,5052.001,2384.373,1,728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30106,14303,1,285001.426,345.456274,1009.576868,1.786,9919.347,0.0,284.0,26.0,...,48914.807,12417.936,36496.871,468,357,155499.239,129502.187,25997.052,0,729
30107,22301,2,34950.321,49.715962,116.602464,0.043,2136.877,2.0,168.0,9.0,...,4142.795,1823.246,2319.549,400,303,21392.851,13557.470,7835.381,0,729
30108,25731,0,54074.454,49.338005,80.490725,0.181,884.970,0.0,266.0,8.0,...,8651.979,2023.383,6628.596,655,441,33673.864,20400.590,13273.274,73,729
30109,16820,3,32494.435,42.643615,136.489534,0.388,2959.186,2.0,340.0,3.0,...,7181.225,1221.195,5960.030,448,314,16554.899,15939.536,615.363,10,729


In [38]:
train[:1].shape

(1, 630)

In [39]:
y_train = train["bins"]
del train["bins"]

In [46]:
import catboost as cb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import xgboost as xgb

In [41]:
def get_stat(data, func='count'):
    if func == "count":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].count()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_'+str(i) for i in cat_counts.columns]
    
    elif func == "sum":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].sum()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_sum_'+str(i) for i in cat_counts.columns]
    
    elif func == 'std':
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].std()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_std_'+str(i) for i in cat_counts.columns]
        
    elif func == "median":
        counter_df = data.groupby(['client_id','small_group'])['amount_rur'].median()
        cat_counts = counter_df.reset_index().pivot(index='client_id', \
                                                      columns='small_group', values='amount_rur')
    
        cat_counts = cat_counts.fillna(0)
        cat_counts.columns = ['small_group_median_'+str(i) for i in cat_counts.columns]
        
    return cat_counts



In [42]:
cat_counts = pd.concat([get_stat(transactions_train, func='count'), get_stat(transactions_train, func='sum'),
                        get_stat(transactions_train, func='std'), get_stat(transactions_train, func='median')],axis=1)

In [43]:
train = pd.merge(train, cat_counts.reset_index(), on='client_id')

In [51]:
train.head(3)

,client_id,sum,mean,std,min,max,sg_count_0,sg_count_1,sg_count_2,sg_count_3,...,small_group_median_192,small_group_median_193,small_group_median_195,small_group_median_196,small_group_median_197,small_group_median_198,small_group_median_199,small_group_median_200,small_group_median_202,small_group_median_203
0,24662,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,64.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1046,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,47.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,34089,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,72.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [52]:
del train["client_id"]

In [53]:
model = cb.CatBoostClassifier(iterations = 5000, learning_rate = 0.04, depth = 5,
                              loss_function = 'MultiClass', eval_metric='Accuracy',
                              random_state = 53, task_type="GPU", devices='0', verbose = 1)

res = cross_validate(model, train, y_train, cv=3, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.5068254	total: 19.8ms	remaining: 1m 38s
1:	learn: 0.5193304	total: 37.8ms	remaining: 1m 34s
2:	learn: 0.5330311	total: 55.8ms	remaining: 1m 32s
3:	learn: 0.5329813	total: 73.2ms	remaining: 1m 31s
4:	learn: 0.5398565	total: 89.8ms	remaining: 1m 29s
5:	learn: 0.5425468	total: 107ms	remaining: 1m 29s
6:	learn: 0.5444898	total: 124ms	remaining: 1m 28s
7:	learn: 0.5465823	total: 141ms	remaining: 1m 28s
8:	learn: 0.5499203	total: 159ms	remaining: 1m 28s
9:	learn: 0.5477780	total: 176ms	remaining: 1m 27s
10:	learn: 0.5502192	total: 193ms	remaining: 1m 27s
11:	learn: 0.5509665	total: 211ms	remaining: 1m 27s
12:	learn: 0.5520626	total: 228ms	remaining: 1m 27s
13:	learn: 0.5545536	total: 244ms	remaining: 1m 27s
14:	learn: 0.5556497	total: 265ms	remaining: 1m 28s
15:	learn: 0.5568454	total: 282ms	remaining: 1m 27s
16:	learn: 0.5589378	total: 298ms	remaining: 1m 27s
17:	learn: 0.5584894	total: 314ms	remaining: 1m 26s
18:	learn: 0.5601833	total: 330ms	remaining: 1m 26s
19:	learn: 0.5635

164:	learn: 0.6339179	total: 2.49s	remaining: 1m 12s
165:	learn: 0.6336190	total: 2.5s	remaining: 1m 12s
166:	learn: 0.6343663	total: 2.52s	remaining: 1m 12s
167:	learn: 0.6341172	total: 2.53s	remaining: 1m 12s
168:	learn: 0.6345656	total: 2.54s	remaining: 1m 12s
169:	learn: 0.6348645	total: 2.56s	remaining: 1m 12s
170:	learn: 0.6347150	total: 2.57s	remaining: 1m 12s
171:	learn: 0.6354623	total: 2.58s	remaining: 1m 12s
172:	learn: 0.6355122	total: 2.6s	remaining: 1m 12s
173:	learn: 0.6357114	total: 2.61s	remaining: 1m 12s
174:	learn: 0.6358111	total: 2.63s	remaining: 1m 12s
175:	learn: 0.6358111	total: 2.64s	remaining: 1m 12s
176:	learn: 0.6358609	total: 2.66s	remaining: 1m 12s
177:	learn: 0.6365086	total: 2.67s	remaining: 1m 12s
178:	learn: 0.6376046	total: 2.68s	remaining: 1m 12s
179:	learn: 0.6380032	total: 2.7s	remaining: 1m 12s
180:	learn: 0.6379035	total: 2.71s	remaining: 1m 12s
181:	learn: 0.6379534	total: 2.73s	remaining: 1m 12s
182:	learn: 0.6383519	total: 2.74s	remaining: 1m 

330:	learn: 0.6641092	total: 4.74s	remaining: 1m 6s
331:	learn: 0.6639597	total: 4.75s	remaining: 1m 6s
332:	learn: 0.6648067	total: 4.76s	remaining: 1m 6s
333:	learn: 0.6647071	total: 4.78s	remaining: 1m 6s
334:	learn: 0.6655540	total: 4.79s	remaining: 1m 6s
335:	learn: 0.6656536	total: 4.8s	remaining: 1m 6s
336:	learn: 0.6657035	total: 4.82s	remaining: 1m 6s
337:	learn: 0.6657035	total: 4.83s	remaining: 1m 6s
338:	learn: 0.6659028	total: 4.85s	remaining: 1m 6s
339:	learn: 0.6660024	total: 4.86s	remaining: 1m 6s
340:	learn: 0.6662017	total: 4.87s	remaining: 1m 6s
341:	learn: 0.6665006	total: 4.88s	remaining: 1m 6s
342:	learn: 0.6664508	total: 4.9s	remaining: 1m 6s
343:	learn: 0.6665006	total: 4.91s	remaining: 1m 6s
344:	learn: 0.6670486	total: 4.92s	remaining: 1m 6s
345:	learn: 0.6670486	total: 4.94s	remaining: 1m 6s
346:	learn: 0.6670486	total: 4.95s	remaining: 1m 6s
347:	learn: 0.6669988	total: 4.97s	remaining: 1m 6s
348:	learn: 0.6671981	total: 4.98s	remaining: 1m 6s
349:	learn: 0.

488:	learn: 0.6844859	total: 6.78s	remaining: 1m 2s
489:	learn: 0.6844859	total: 6.79s	remaining: 1m 2s
490:	learn: 0.6845855	total: 6.81s	remaining: 1m 2s
491:	learn: 0.6849841	total: 6.82s	remaining: 1m 2s
492:	learn: 0.6851335	total: 6.84s	remaining: 1m 2s
493:	learn: 0.6852332	total: 6.85s	remaining: 1m 2s
494:	learn: 0.6852332	total: 6.86s	remaining: 1m 2s
495:	learn: 0.6848844	total: 6.88s	remaining: 1m 2s
496:	learn: 0.6849841	total: 6.89s	remaining: 1m 2s
497:	learn: 0.6856317	total: 6.9s	remaining: 1m 2s
498:	learn: 0.6858310	total: 6.91s	remaining: 1m 2s
499:	learn: 0.6859805	total: 6.92s	remaining: 1m 2s
500:	learn: 0.6858808	total: 6.94s	remaining: 1m 2s
501:	learn: 0.6860801	total: 6.95s	remaining: 1m 2s
502:	learn: 0.6862794	total: 6.96s	remaining: 1m 2s
503:	learn: 0.6862296	total: 6.97s	remaining: 1m 2s
504:	learn: 0.6860303	total: 6.99s	remaining: 1m 2s
505:	learn: 0.6863292	total: 7s	remaining: 1m 2s
506:	learn: 0.6862794	total: 7.01s	remaining: 1m 2s
507:	learn: 0.68

...

4422:	learn: 0.8634688	total: 53.8s	remaining: 7.02s
4423:	learn: 0.8635186	total: 53.8s	remaining: 7s
4424:	learn: 0.8635186	total: 53.8s	remaining: 6.99s
4425:	learn: 0.8633692	total: 53.8s	remaining: 6.98s
4426:	learn: 0.8638175	total: 53.8s	remaining: 6.97s
4427:	learn: 0.8636681	total: 53.9s	remaining: 6.96s
4428:	learn: 0.8635684	total: 53.9s	remaining: 6.94s
4429:	learn: 0.8635684	total: 53.9s	remaining: 6.93s
4430:	learn: 0.8636183	total: 53.9s	remaining: 6.92s
4431:	learn: 0.8636183	total: 53.9s	remaining: 6.91s
4432:	learn: 0.8637677	total: 53.9s	remaining: 6.89s
4433:	learn: 0.8639171	total: 53.9s	remaining: 6.88s
4434:	learn: 0.8640665	total: 53.9s	remaining: 6.87s
4435:	learn: 0.8640167	total: 53.9s	remaining: 6.86s
4436:	learn: 0.8640167	total: 54s	remaining: 6.85s
4437:	learn: 0.8639171	total: 54s	remaining: 6.83s
4438:	learn: 0.8639171	total: 54s	remaining: 6.82s
4439:	learn: 0.8640665	total: 54s	remaining: 6.81s
4440:	learn: 0.8641164	total: 54s	remaining: 6.8s
4441:	l

4584:	learn: 0.8683005	total: 55.6s	remaining: 5.04s
4585:	learn: 0.8683503	total: 55.7s	remaining: 5.02s
4586:	learn: 0.8681510	total: 55.7s	remaining: 5.01s
4587:	learn: 0.8681510	total: 55.7s	remaining: 5s
4588:	learn: 0.8681012	total: 55.7s	remaining: 4.99s
4589:	learn: 0.8682008	total: 55.7s	remaining: 4.97s
4590:	learn: 0.8683005	total: 55.7s	remaining: 4.96s
4591:	learn: 0.8682506	total: 55.7s	remaining: 4.95s
4592:	learn: 0.8683005	total: 55.7s	remaining: 4.94s
4593:	learn: 0.8682506	total: 55.7s	remaining: 4.92s
4594:	learn: 0.8682506	total: 55.7s	remaining: 4.91s
4595:	learn: 0.8681510	total: 55.8s	remaining: 4.9s
4596:	learn: 0.8682008	total: 55.8s	remaining: 4.89s
4597:	learn: 0.8682008	total: 55.8s	remaining: 4.88s
4598:	learn: 0.8683005	total: 55.8s	remaining: 4.86s
4599:	learn: 0.8683005	total: 55.8s	remaining: 4.85s
4600:	learn: 0.8678522	total: 55.8s	remaining: 4.84s
4601:	learn: 0.8681012	total: 55.8s	remaining: 4.83s
4602:	learn: 0.8680514	total: 55.8s	remaining: 4.8

4747:	learn: 0.8721857	total: 57.5s	remaining: 3.05s
4748:	learn: 0.8721857	total: 57.5s	remaining: 3.04s
4749:	learn: 0.8721359	total: 57.5s	remaining: 3.03s
4750:	learn: 0.8720363	total: 57.5s	remaining: 3.01s
4751:	learn: 0.8720363	total: 57.5s	remaining: 3s
4752:	learn: 0.8722853	total: 57.5s	remaining: 2.99s
4753:	learn: 0.8722355	total: 57.6s	remaining: 2.98s
4754:	learn: 0.8722853	total: 57.6s	remaining: 2.96s
4755:	learn: 0.8722355	total: 57.6s	remaining: 2.95s
4756:	learn: 0.8721857	total: 57.6s	remaining: 2.94s
4757:	learn: 0.8721857	total: 57.6s	remaining: 2.93s
4758:	learn: 0.8721857	total: 57.6s	remaining: 2.92s
4759:	learn: 0.8722853	total: 57.6s	remaining: 2.9s
4760:	learn: 0.8724347	total: 57.6s	remaining: 2.89s
4761:	learn: 0.8722853	total: 57.6s	remaining: 2.88s
4762:	learn: 0.8721857	total: 57.6s	remaining: 2.87s
4763:	learn: 0.8722853	total: 57.7s	remaining: 2.86s
4764:	learn: 0.8719865	total: 57.7s	remaining: 2.84s
4765:	learn: 0.8719865	total: 57.7s	remaining: 2.8

4912:	learn: 0.8757223	total: 59.3s	remaining: 1.05s
4913:	learn: 0.8757223	total: 59.3s	remaining: 1.04s
4914:	learn: 0.8756226	total: 59.4s	remaining: 1.03s
4915:	learn: 0.8756724	total: 59.4s	remaining: 1.01s
4916:	learn: 0.8756724	total: 59.4s	remaining: 1s
4917:	learn: 0.8756226	total: 59.4s	remaining: 990ms
4918:	learn: 0.8756226	total: 59.4s	remaining: 978ms
4919:	learn: 0.8757223	total: 59.4s	remaining: 966ms
4920:	learn: 0.8757223	total: 59.4s	remaining: 954ms
4921:	learn: 0.8755230	total: 59.4s	remaining: 942ms
4922:	learn: 0.8755230	total: 59.4s	remaining: 930ms
4923:	learn: 0.8754234	total: 59.5s	remaining: 918ms
4924:	learn: 0.8754732	total: 59.5s	remaining: 905ms
4925:	learn: 0.8755728	total: 59.5s	remaining: 893ms
4926:	learn: 0.8757721	total: 59.5s	remaining: 881ms
4927:	learn: 0.8758219	total: 59.5s	remaining: 869ms
4928:	learn: 0.8758219	total: 59.5s	remaining: 857ms
4929:	learn: 0.8759215	total: 59.5s	remaining: 845ms
4930:	learn: 0.8759215	total: 59.5s	remaining: 83

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  3.1min finished


In [54]:
np.mean(res['test_score'])

0.6405302278638166

In [55]:
res['test_score']

array([0.63890826, 0.64092856, 0.64175386])

In [56]:
model.fit(train, y_train)

0:	learn: 0.5094484	total: 22ms	remaining: 1m 49s
1:	learn: 0.5222012	total: 41ms	remaining: 1m 42s
2:	learn: 0.5161901	total: 60.2ms	remaining: 1m 40s
3:	learn: 0.5245591	total: 79.6ms	remaining: 1m 39s
4:	learn: 0.5287769	total: 99ms	remaining: 1m 38s
5:	learn: 0.5326957	total: 119ms	remaining: 1m 39s
6:	learn: 0.5408655	total: 138ms	remaining: 1m 38s
7:	learn: 0.5431902	total: 158ms	remaining: 1m 38s
8:	learn: 0.5420610	total: 178ms	remaining: 1m 38s
9:	learn: 0.5413636	total: 197ms	remaining: 1m 38s
10:	learn: 0.5428249	total: 216ms	remaining: 1m 37s
11:	learn: 0.5454817	total: 234ms	remaining: 1m 37s
12:	learn: 0.5480389	total: 254ms	remaining: 1m 37s
13:	learn: 0.5493673	total: 278ms	remaining: 1m 38s
14:	learn: 0.5516921	total: 296ms	remaining: 1m 38s
15:	learn: 0.5534854	total: 314ms	remaining: 1m 37s
16:	learn: 0.5542161	total: 331ms	remaining: 1m 36s
17:	learn: 0.5548139	total: 349ms	remaining: 1m 36s
18:	learn: 0.5550795	total: 367ms	remaining: 1m 36s
19:	learn: 0.5568397	to

158:	learn: 0.6241241	total: 2.7s	remaining: 1m 22s
159:	learn: 0.6244894	total: 2.72s	remaining: 1m 22s
160:	learn: 0.6245558	total: 2.74s	remaining: 1m 22s
161:	learn: 0.6247551	total: 2.76s	remaining: 1m 22s
162:	learn: 0.6249211	total: 2.79s	remaining: 1m 22s
163:	learn: 0.6247883	total: 2.81s	remaining: 1m 22s
164:	learn: 0.6253197	total: 2.83s	remaining: 1m 22s
165:	learn: 0.6250872	total: 2.85s	remaining: 1m 22s
166:	learn: 0.6254857	total: 2.87s	remaining: 1m 23s
167:	learn: 0.6259506	total: 2.89s	remaining: 1m 23s
168:	learn: 0.6263824	total: 2.91s	remaining: 1m 23s
169:	learn: 0.6266813	total: 2.94s	remaining: 1m 23s
170:	learn: 0.6268473	total: 2.96s	remaining: 1m 23s
171:	learn: 0.6272126	total: 2.98s	remaining: 1m 23s
172:	learn: 0.6272459	total: 3s	remaining: 1m 23s
173:	learn: 0.6278436	total: 3.02s	remaining: 1m 23s
174:	learn: 0.6282754	total: 3.04s	remaining: 1m 23s
175:	learn: 0.6288732	total: 3.06s	remaining: 1m 23s
176:	learn: 0.6282090	total: 3.08s	remaining: 1m 2

313:	learn: 0.6499950	total: 5.36s	remaining: 1m 19s
314:	learn: 0.6501943	total: 5.37s	remaining: 1m 19s
315:	learn: 0.6507589	total: 5.39s	remaining: 1m 19s
316:	learn: 0.6508585	total: 5.41s	remaining: 1m 19s
317:	learn: 0.6512238	total: 5.42s	remaining: 1m 19s
318:	learn: 0.6514563	total: 5.43s	remaining: 1m 19s
319:	learn: 0.6516223	total: 5.45s	remaining: 1m 19s
320:	learn: 0.6516555	total: 5.46s	remaining: 1m 19s
321:	learn: 0.6518548	total: 5.48s	remaining: 1m 19s
322:	learn: 0.6517884	total: 5.49s	remaining: 1m 19s
323:	learn: 0.6517552	total: 5.51s	remaining: 1m 19s
324:	learn: 0.6518548	total: 5.53s	remaining: 1m 19s
325:	learn: 0.6521537	total: 5.54s	remaining: 1m 19s
326:	learn: 0.6522201	total: 5.56s	remaining: 1m 19s
327:	learn: 0.6523198	total: 5.57s	remaining: 1m 19s
328:	learn: 0.6525190	total: 5.59s	remaining: 1m 19s
329:	learn: 0.6524858	total: 5.6s	remaining: 1m 19s
330:	learn: 0.6527515	total: 5.62s	remaining: 1m 19s
331:	learn: 0.6530504	total: 5.63s	remaining: 1

469:	learn: 0.6676630	total: 7.63s	remaining: 1m 13s
470:	learn: 0.6675301	total: 7.65s	remaining: 1m 13s
471:	learn: 0.6678290	total: 7.66s	remaining: 1m 13s
472:	learn: 0.6675966	total: 7.67s	remaining: 1m 13s
473:	learn: 0.6677626	total: 7.69s	remaining: 1m 13s
474:	learn: 0.6679287	total: 7.7s	remaining: 1m 13s
475:	learn: 0.6680615	total: 7.72s	remaining: 1m 13s
476:	learn: 0.6679951	total: 7.73s	remaining: 1m 13s
477:	learn: 0.6680947	total: 7.75s	remaining: 1m 13s
478:	learn: 0.6681279	total: 7.76s	remaining: 1m 13s
479:	learn: 0.6680283	total: 7.78s	remaining: 1m 13s
480:	learn: 0.6683604	total: 7.79s	remaining: 1m 13s
481:	learn: 0.6686925	total: 7.81s	remaining: 1m 13s
482:	learn: 0.6689914	total: 7.82s	remaining: 1m 13s
483:	learn: 0.6689914	total: 7.83s	remaining: 1m 13s
484:	learn: 0.6689914	total: 7.85s	remaining: 1m 13s
485:	learn: 0.6691242	total: 7.86s	remaining: 1m 13s
486:	learn: 0.6691907	total: 7.88s	remaining: 1m 12s
487:	learn: 0.6692571	total: 7.89s	remaining: 1

630:	learn: 0.6799508	total: 9.91s	remaining: 1m 8s
631:	learn: 0.6799508	total: 9.92s	remaining: 1m 8s
632:	learn: 0.6800173	total: 9.93s	remaining: 1m 8s
633:	learn: 0.6802165	total: 9.95s	remaining: 1m 8s
634:	learn: 0.6801501	total: 9.96s	remaining: 1m 8s
635:	learn: 0.6803826	total: 9.98s	remaining: 1m 8s
636:	learn: 0.6801501	total: 9.99s	remaining: 1m 8s
637:	learn: 0.6803162	total: 10s	remaining: 1m 8s
638:	learn: 0.6802165	total: 10s	remaining: 1m 8s
639:	learn: 0.6807147	total: 10s	remaining: 1m 8s
640:	learn: 0.6806815	total: 10s	remaining: 1m 8s
641:	learn: 0.6807479	total: 10.1s	remaining: 1m 8s
642:	learn: 0.6805818	total: 10.1s	remaining: 1m 8s
643:	learn: 0.6808143	total: 10.1s	remaining: 1m 8s
644:	learn: 0.6810468	total: 10.1s	remaining: 1m 8s
645:	learn: 0.6811796	total: 10.1s	remaining: 1m 8s
646:	learn: 0.6811796	total: 10.1s	remaining: 1m 8s
647:	learn: 0.6810136	total: 10.1s	remaining: 1m 8s
648:	learn: 0.6809140	total: 10.2s	remaining: 1m 8s
649:	learn: 0.681179

...

4185:	learn: 0.8121617	total: 55.8s	remaining: 10.8s
4186:	learn: 0.8121949	total: 55.8s	remaining: 10.8s
4187:	learn: 0.8122281	total: 55.8s	remaining: 10.8s
4188:	learn: 0.8122613	total: 55.8s	remaining: 10.8s
4189:	learn: 0.8123941	total: 55.8s	remaining: 10.8s
4190:	learn: 0.8124938	total: 55.8s	remaining: 10.8s
4191:	learn: 0.8123609	total: 55.9s	remaining: 10.8s
4192:	learn: 0.8123941	total: 55.9s	remaining: 10.8s
4193:	learn: 0.8122613	total: 55.9s	remaining: 10.7s
4194:	learn: 0.8122613	total: 55.9s	remaining: 10.7s
4195:	learn: 0.8123277	total: 55.9s	remaining: 10.7s
4196:	learn: 0.8123277	total: 55.9s	remaining: 10.7s
4197:	learn: 0.8121617	total: 55.9s	remaining: 10.7s
4198:	learn: 0.8122945	total: 55.9s	remaining: 10.7s
4199:	learn: 0.8121949	total: 56s	remaining: 10.7s
4200:	learn: 0.8122613	total: 56s	remaining: 10.6s
4201:	learn: 0.8122945	total: 56s	remaining: 10.6s
4202:	learn: 0.8124274	total: 56s	remaining: 10.6s
4203:	learn: 0.8122945	total: 56s	remaining: 10.6s
420

4345:	learn: 0.8149846	total: 57.8s	remaining: 8.7s
4346:	learn: 0.8149846	total: 57.8s	remaining: 8.69s
4347:	learn: 0.8149846	total: 57.8s	remaining: 8.67s
4348:	learn: 0.8149846	total: 57.9s	remaining: 8.66s
4349:	learn: 0.8149846	total: 57.9s	remaining: 8.65s
4350:	learn: 0.8149513	total: 57.9s	remaining: 8.63s
4351:	learn: 0.8149846	total: 57.9s	remaining: 8.62s
4352:	learn: 0.8150842	total: 57.9s	remaining: 8.61s
4353:	learn: 0.8152170	total: 57.9s	remaining: 8.59s
4354:	learn: 0.8151506	total: 57.9s	remaining: 8.58s
4355:	learn: 0.8151838	total: 57.9s	remaining: 8.57s
4356:	learn: 0.8151174	total: 58s	remaining: 8.55s
4357:	learn: 0.8153167	total: 58s	remaining: 8.54s
4358:	learn: 0.8152502	total: 58s	remaining: 8.53s
4359:	learn: 0.8152502	total: 58s	remaining: 8.51s
4360:	learn: 0.8152502	total: 58s	remaining: 8.5s
4361:	learn: 0.8152170	total: 58s	remaining: 8.49s
4362:	learn: 0.8151838	total: 58s	remaining: 8.47s
4363:	learn: 0.8151506	total: 58s	remaining: 8.46s
4364:	learn

4507:	learn: 0.8184717	total: 59.9s	remaining: 6.54s
4508:	learn: 0.8185049	total: 59.9s	remaining: 6.52s
4509:	learn: 0.8186377	total: 59.9s	remaining: 6.51s
4510:	learn: 0.8186045	total: 59.9s	remaining: 6.5s
4511:	learn: 0.8187041	total: 59.9s	remaining: 6.48s
4512:	learn: 0.8187373	total: 59.9s	remaining: 6.47s
4513:	learn: 0.8188038	total: 60s	remaining: 6.46s
4514:	learn: 0.8188038	total: 60s	remaining: 6.44s
4515:	learn: 0.8189698	total: 60s	remaining: 6.43s
4516:	learn: 0.8189698	total: 60s	remaining: 6.42s
4517:	learn: 0.8189034	total: 1m	remaining: 6.4s
4518:	learn: 0.8189034	total: 1m	remaining: 6.39s
4519:	learn: 0.8188702	total: 1m	remaining: 6.38s
4520:	learn: 0.8188370	total: 1m	remaining: 6.36s
4521:	learn: 0.8189034	total: 1m	remaining: 6.35s
4522:	learn: 0.8190694	total: 1m	remaining: 6.33s
4523:	learn: 0.8189698	total: 1m	remaining: 6.32s
4524:	learn: 0.8190030	total: 1m	remaining: 6.31s
4525:	learn: 0.8190694	total: 1m	remaining: 6.3s
4526:	learn: 0.8191027	total: 1

4682:	learn: 0.8222909	total: 1m 2s	remaining: 4.2s
4683:	learn: 0.8224237	total: 1m 2s	remaining: 4.19s
4684:	learn: 0.8224901	total: 1m 2s	remaining: 4.18s
4685:	learn: 0.8226894	total: 1m 2s	remaining: 4.17s
4686:	learn: 0.8227226	total: 1m 2s	remaining: 4.15s
4687:	learn: 0.8227226	total: 1m 2s	remaining: 4.14s
4688:	learn: 0.8226894	total: 1m 2s	remaining: 4.13s
4689:	learn: 0.8226894	total: 1m 2s	remaining: 4.11s
4690:	learn: 0.8228222	total: 1m 2s	remaining: 4.1s
4691:	learn: 0.8228554	total: 1m 2s	remaining: 4.08s
4692:	learn: 0.8228554	total: 1m 2s	remaining: 4.07s
4693:	learn: 0.8229551	total: 1m 2s	remaining: 4.06s
4694:	learn: 0.8228886	total: 1m 2s	remaining: 4.04s
4695:	learn: 0.8228554	total: 1m 2s	remaining: 4.03s
4696:	learn: 0.8230879	total: 1m 2s	remaining: 4.02s
4697:	learn: 0.8229551	total: 1m 2s	remaining: 4s
4698:	learn: 0.8229219	total: 1m 2s	remaining: 3.99s
4699:	learn: 0.8230547	total: 1m 2s	remaining: 3.98s
4700:	learn: 0.8230879	total: 1m 2s	remaining: 3.96

4842:	learn: 0.8273388	total: 1m 4s	remaining: 2.08s
4843:	learn: 0.8273056	total: 1m 4s	remaining: 2.07s
4844:	learn: 0.8272392	total: 1m 4s	remaining: 2.05s
4845:	learn: 0.8271728	total: 1m 4s	remaining: 2.04s
4846:	learn: 0.8273721	total: 1m 4s	remaining: 2.03s
4847:	learn: 0.8274053	total: 1m 4s	remaining: 2.01s
4848:	learn: 0.8273056	total: 1m 4s	remaining: 2s
4849:	learn: 0.8273388	total: 1m 4s	remaining: 1.99s
4850:	learn: 0.8275049	total: 1m 4s	remaining: 1.97s
4851:	learn: 0.8275381	total: 1m 4s	remaining: 1.96s
4852:	learn: 0.8275381	total: 1m 4s	remaining: 1.95s
4853:	learn: 0.8275713	total: 1m 4s	remaining: 1.93s
4854:	learn: 0.8275049	total: 1m 4s	remaining: 1.92s
4855:	learn: 0.8275713	total: 1m 4s	remaining: 1.91s
4856:	learn: 0.8275381	total: 1m 4s	remaining: 1.89s
4857:	learn: 0.8275049	total: 1m 4s	remaining: 1.88s
4858:	learn: 0.8275049	total: 1m 4s	remaining: 1.87s
4859:	learn: 0.8275381	total: 1m 4s	remaining: 1.85s
4860:	learn: 0.8274385	total: 1m 4s	remaining: 1.

In [58]:
model.feature_importances_

array([0.38326924, 1.50469307, 0.21136692, ..., 0.        , 0.        ,
       0.        ])

In [62]:
import eli5
eli5.catboost.explain_weights(model, top=1000)

Weight,Feature
0.0335,small_group_median_1
0.0206,small_group_sum_1
0.0183,sg_count_11
0.0181,"sg_sum_('sum', 1)"
0.0165,sg_count_12
0.0155,yearcount_min
0.0150,mean
0.0149,small_group_median_25
0.0102,"sg_sum_('sum', 3)"
0.0096,small_group_median_11


In [60]:
cb.FeaturesData?

In [47]:
param={'objective':'multi:softprob','num_class':4,'n_jobs':16,'seed':42}
model_xgb = xgb.XGBClassifier(**param,n_estimators=600)

In [48]:
res_xgb = cross_validate(model_xgb, train, y_train, cv=3, verbose=1)
np.mean(res_xgb['test_score'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  6.4min finished


0.6293052063811585

In [50]:
res

{'fit_time': array([61.10578203, 62.17099953, 62.10314608]),
 'score_time': array([0.11811471, 0.10863566, 0.11922956]),
 'test_score': array([0.6367168 , 0.64401714, 0.64205282])}

In [110]:
np.mean([0.62350598, 0.625     , 0.62189451, 0.63114536])

0.6253864625

In [86]:
np.mean([0.62124834, 0.62207758, 0.62269164, 0.6290194 ])

0.62375924

In [57]:
np.mean([0.61341301, 0.6186238 , 0.61525176, 0.61506777])

0.615589085

In [32]:
np.mean([0.61580345, 0.61517003, 0.6124618 , 0.61639649])

0.6149579425

In [31]:
np.mean([0.61383631, 0.61813333, 0.61154821, 0.61594879])

0.61486666

# Знаю как реализовать
* - Максимальная транзакция
* - КАтегория максимальной транзакции
* Количество транзакций ниже трешхолда
* Трешхолд для каждого пользователя свой (типо выше среднего)
* Уровень трат выше среднего уровня трат для каждой категории
* Максимальные затраты за 1 день
* Сколько в среднем тратит за 1 день
* Производная от графика (затраты / дни)

# Надо думать

* как считается trans_date ? (Относительно получения карты или абсолютно)
* если абсолютно, то можно попытаться определить дни недели